Copyright **`(c)`** 2022 Giovanni Squillero `<squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  


# Lab 3: ES

## Task

Write agents able to play [*Nim*](https://en.wikipedia.org/wiki/Nim), with an arbitrary number of rows and an upper bound $k$ on the number of objects that can be removed in a turn (a.k.a., *subtraction game*).

The goal of the game is to **avoid** taking the last object.

* Task2.1: An agent using fixed rules based on *nim-sum* (i.e., an *expert system*)
* Task2.2: An agent using evolved rules using ES

## Instructions

* Create the directory `lab2` inside the course repo 
* Put a `README.md` and your solution (all the files, code and auxiliary data if needed)

## Notes

* Working in group is not only allowed, but recommended (see: [Ubuntu](https://en.wikipedia.org/wiki/Ubuntu_philosophy) and [Cooperative Learning](https://files.eric.ed.gov/fulltext/EJ1096789.pdf)). Collaborations must be explicitly declared in the `README.md`.
* [Yanking](https://www.emacswiki.org/emacs/KillingAndYanking) from the internet is allowed, but sources must be explicitly declared in the `README.md`.



In [60]:
import logging
from pprint import pprint, pformat
from collections import namedtuple
import random
from copy import deepcopy


## The *Nim* and *Nimply* classes

In [61]:
Nimply = namedtuple("Nimply", "row, num_objects")


In [62]:
class Nim:
    def __init__(self, num_rows: int, k: int = None) -> None:
        self._rows = [i * 2 + 1 for i in range(num_rows)]
        self._k = k

    def __bool__(self):
        return sum(self._rows) > 0

    def __str__(self):
        return "<" + " ".join(str(_) for _ in self._rows) + ">"

    @property
    def rows(self) -> tuple:
        return tuple(self._rows)

    def nimming(self, ply: Nimply) -> None:
        row, num_objects = ply
        assert self._rows[row] >= num_objects
        assert self._k is None or num_objects <= self._k
        self._rows[row] -= num_objects


## Sample (and silly) startegies 

In [63]:
def pure_random(state: Nim) -> Nimply:
    """A completely random move"""
    row = random.choice([r for r, c in enumerate(state.rows) if c > 0])
    num_objects = random.randint(1, state.rows[row])
    return Nimply(row, num_objects)


In [64]:
def gabriele(state: Nim) -> Nimply:
    """Pick always the maximum possible number of the lowest row"""
    possible_moves = [(r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1)]
    return Nimply(*max(possible_moves, key=lambda m: (-m[0], m[1])))


In [65]:
def adaptive(state: Nim) -> Nimply:
    """A strategy that can adapt its parameters"""
    genome = {"love_small": 0.5}


In [66]:
import numpy as np

# converts each row in a binary number at 32 bit and it returns a binary array with the sum of all the rows
def nim_sum(state: Nim) -> int:
    tmp = np.array([tuple(int(x) for x in f"{c:032b}") for c in state.rows])
    xor = tmp.sum(axis=0) % 2
    return int("".join(str(_) for _ in xor), base=2)


def analize(raw: Nim) -> dict:
    cooked = dict()
    cooked["possible_moves"] = dict()
    for ply in (Nimply(r, o) for r, c in enumerate(raw.rows) for o in range(1, c + 1)):
        tmp = deepcopy(raw)
        tmp.nimming(ply)
        cooked["possible_moves"][ply] = nim_sum(tmp)
    return cooked



def optimal(state: Nim) -> Nimply:
    analysis = analize(state)
    logging.debug(f"analysis:\n{pformat(analysis)}")
    spicy_moves = [ply for ply, ns in analysis["possible_moves"].items() if ns != 0]
    if not spicy_moves:
        spicy_moves = list(analysis["possible_moves"].keys())
    ply = random.choice(spicy_moves)
    return ply

## Oversimplified match

In [ ]:
logging.getLogger().setLevel(logging.INFO)

strategy = (optimal, pure_random)

nim = Nim(5)
logging.info(f"init : {nim}")
player = 0
while nim:
    ply = strategy[player](nim)
    logging.info(f"ply: player {player} plays {ply}")
    nim.nimming(ply)
    logging.info(f"status: {nim}")
    player = 1 - player
logging.info(f"status: Player {player} won!")


# Solution using ES

In [ ]:
POPULATION_SIZE = 30
OFFSPRING_SIZE = 20
ROUNDS = 10
NUM_ROWS = 5
TOURNAMENT_SIZE = 2
MUTATION_PROBABILITY = .15

In [68]:
def fitness_function():
    total_remaining_sticks = 0

    for _ in range(ROUNDS):
        nim = Nim(NUM_ROWS)
        player = 0

        while nim:
            ply = strategy[player](nim)
            nim.nimming(ply)
            player = 1 - player

        total_remaining_sticks += sum(nim.rows)

    # The fitness is the average number of remaining sticks over multiple rounds
    average_remaining_sticks = total_remaining_sticks / ROUNDS
    return average_remaining_sticks

def tournament_selection(population, fitness_values, tournament_size):
    selected_indices = random.sample(range(len(population)), tournament_size)
    tournament_fitness = [fitness_values[i] for i in selected_indices]
    winner_index = selected_indices[tournament_fitness.index(max(tournament_fitness))]
    return population[winner_index]

def select_parent(pop):
    pool = [random.choice(pop) for _ in range(TOURNAMENT_SIZE)]
    champion = max(pool, )
    return champion

def mutate(ind: Individual) -> Individual:
    offspring = copy(ind)
    pos = randint(0, NUM_SETS-1)
    offspring.genotype[pos] = not offspring.genotype[pos]
    offspring.fitness = None
    return offspring

def one_cut_xover(ind1: Individual, ind2: Individual) -> Individual:
    cut_point = randint(0, NUM_SETS-1)
    offspring = Individual(fitness=None,
                           genotype=ind1.genotype[:cut_point] + ind2.genotype[cut_point:])
    assert len(offspring.genotype) == NUM_SETS
    return offspring

def evolutionary_algorithm(population_size, generations, rounds_per_game):
    population = [random.choiche([optimal, pure_random]) for _ in range(population_size)]

    for generation in range(generations):
        fitness_values = [fitness_function(individual, rounds_per_game) for individual in population]

        # Select parents using tournament selection and create offspring through crossover and mutation
        new_population = []
        for _ in range(population_size // 2):
            parent1 = select_parent(population, fitness_values)
            parent2 = select_parent(population, fitness_values)
            child1 = mutate(parent1)
            child2 = mutate(parent2)
            new_population.extend([child1, child2])

        population = new_population

        # Print the best strategy in each generation
        best_strategy = population[fitness_values.index(max(fitness_values))]
        print(f"Generation {generation + 1}: Best Fitness = {fitness_function(best_strategy, rounds_per_game)}")

    # Return the best strategy at the end of the evolution
    return population[fitness_values.index(max(fitness_values))]